In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import polars as pl
import requests
import bambi as bmb
import geopandas as gpd
from pysal.lib import weights
from shapely import wkt
import pandas as pd
import arviz as az
import matplotlib.dates as mdates
import numpy as np
import causalpy as cp
import matplotlib.pyplot as plt
dr = DataReg()

In [ ]:
df_qcew = dr.base_data().with_columns(
    treatment=pl.when(pl.col("year") >= 2023).then(True).otherwise(False)
)
# df_dp03 = dr.pull_dp03()
pr_zips = gpd.GeoDataFrame(dr.make_spatial_table().df())
pr_zips["geometry"] = pr_zips["geometry"].apply(wkt.loads)
pr_zips = pr_zips.set_geometry("geometry")
pr_zips["zipcode"] = pr_zips["zipcode"].astype(str)

# df = df_qcew.join(df_dp03, on=["zipcode","year"], how="inner")
df = pr_zips.join(
    df_qcew.to_pandas().set_index("zipcode"), on="zipcode", how="inner", validate="1:m"
        ).reset_index(drop=True)
df = df.sort_values("year").reset_index(drop=True)
df

In [ ]:
# sector_df = df[(df["sector"] == "61")].reset_index(drop=True)
y = df["total_employment"].values.reshape(-1,1)
w = weights.KNN.from_dataframe(df, k=5)
reg = df
reg["w_emplyment"] = weights.lag_spatial(w,y) 
reg

In [ ]:
# Assuming `reg` is your DataFrame and you want to set the 'year' column as a datetime index
data = reg.copy()
data = data.drop("geometry", axis=1)
# data = data.drop("sector", axis=1)
# data = data.drop("treatment", axis=1)
data['date'] = data['year'] * 10 + data['qtr']
data

In [ ]:
model = bmb.Model("total_employment ~ k_index + date + (1 + k_index|zipcode) + w_emplyment", data)
results = model.fit()

In [ ]:
model.plot_priors()

In [ ]:
# Plot posteriors
az.plot_trace(
    results,
    compact=True,
)

In [ ]:
ppc = model.predict(results, kind="pps", inplace=False)
az.plot_ppc(ppc)


In [ ]:
az.summary(results)

In [ ]:
ppc = model.predict(results, kind="pps", inplace=False)

# Compute Bayesian R² manually from observed and predicted values
r2_vals = az.r2_score(y=ppc.observed_data["total_employment"],
                      y_hat=ppc.posterior_predictive["total_employment"])

print(r2_vals.mean())  